# Ownership

* Memory is managed through a system of **ownership** with a set of rules that the compiler checks. 
* If any of the rules are violated, the program won’t compile. 
* None of the features of ownership will slow down your program while it’s running.

## Ownership rules

* Each value in Rust has an owner.
* There can only be one owner at a time.
* When the owner goes out of scope, the value will be dropped.

## Variable scope

* The **variable** is valid from the point at which it’s declared until the end of the current **scope**
* There are two important points in time here:
  * when `s` comes into scope, it is valid.
  * it remains valid until it goes out of scope.

In [9]:
{
    let mut s = String::from("Rusty"); // s allocates memory on the heap
    s.push_str(" Cage!"); // ... do something with 's'
    println!("{}", s);
    
} // 's' goes out of scope, and memory is deallocated

Rusty Cage!


## Copy & Move for objects

### Copy

* If a type implements the `Copy` trait, variables that use it do not move, but rather are trivially copied, making them still valid after assignment to another variable.

* Primitive types that implement `Copy` trait are:
  * all the integer types, such as `u32`.
  * the Boolean type, `bool`, with values `true` and `false`.
  * all the floating-point types, such as `f64`.
  * the character type, `char`.
  * tuples, if they only contain types that also implement `Copy`. For example, `(i32, i32)` implements `Copy`, but `(i32, String)` does not.


In [2]:
let x = 10;
let y = x;

println!("x={x}; y={y}");

x=10; y=10


### Move

* Objects that allocate memory on heap are moved by default (shallow copy is disabled by default)

In [15]:
let s1 = String::from("Rusty");
let s2 = s1;

println!("{s2}");

Rusty


In [17]:
let s1 = String::from("Rusty");
let s2 = s1;

s1

Error: borrow of moved value: `s1`

* How it works

<img src="images/rust-move.png" alt="rust-move" height="400" align="center" />

* if we do want to make a deep copy the heap data of the `String`, not just the stack data, we can use a common method called `clone()`.

In [10]:
let s1 = String::from("hello");
let s2 = s1.clone();

println!("s1 = {s1}, s2 = {s2}");

s1 = hello, s2 = hello


<img src="images/rust-clone.png" alt="rust-clone" height="400" align="center" />

#### Move and Indexed Content

* When we move a vector, the vector itself is moved, but its indexed content cannot be moved.

In [6]:
let mut words = vec!["hello".to_string(), "world".to_string(), "!!!".to_string()];

let first_word = words[0];
let second_word = words[1];

Error: cannot move out of index of `Vec<String>`

Error: cannot move out of index of `Vec<String>`

* We can move (consume) items of a vector in a loop:

In [7]:
let mut target_words: Vec<String> = Vec::new();

let mut words = vec!["hello".to_string(), "world".to_string(), "!!!".to_string()];

for word in words {
    target_words.push(word);
}

println!("{:?}", target_words);

["hello", "world", "!!!"]


## Ownership and functions

* The mechanics of passing a value to a function are similar to those when assigning a value to a variable
* Passing a variable to a function will move or copy, just as assignment does

In [ ]:
fn takes_ownership(text: String) {
    println!("text = {text}");
} 

fn takes_copy(number: f64) {
    println!("number = {number}");
}

fn main() {
    let pi = 3.14159;
    takes_copy(pi);
    println!("pi = {pi}");

    let text = String::from("TEXT");
    takes_ownership(text);
    // println!("text = {text}"); // comment-out this line
}

main();

number = 3.14159
pi = 3.14159
text = TEXT


## Return Values and Scope

* Returning values can also transfer ownership

In [17]:
fn gives_ownership() -> String {
    let text = String::from("Ownership!!!");

    text
}

fn transfers_ownership(text: String) -> String {
    text
}

fn main() {
    let txt1 = gives_ownership();
    let txt2 = transfers_ownership(txt1);
    // println!("{txt1}"); // comment-out this line
    println!("{txt2}");

    println!("{}", transfers_ownership(gives_ownership()));    
}

main();

Ownership!!!
Ownership!!!


# References & Borrowing

* A reference lets you access a value without affecting its ownership. References come in two kinds:
  * A **shared reference** lets you read but not modify its referent. However, you can have as many shared references to a particular value at a time as you like. The expression `&e` yields a shared reference to `e`’s value; if `e` has the type `T`, then `&e` has the type `&T`, pronounced “ref T”. Shared references are `Copy`
  * If you have a **mutable reference** to a value, you may both read and modify the value. However, you may not have any other references of any sort to that value active at the same time. The expression `&mut e` yields a mutable reference to `e`’s value; you write its type as `&mut T`, which is pronounced “ref mute T”. Mutable references are not `Copy`.

* In Rust, references are created explicitly with the `&` operator, and dereferenced explicitly with the `*` operator:

In [3]:
{
    let x = 10;
    let rx = &x;        // &x is a shared reference to x
    
    assert!(*rx == 10); // explcitly dereference rx
}

()

* To create a mutable reference, use the `&mut` operator:

In [5]:
{
    let mut y = 42;
    let ry = &mut y;  // &mut y is a mutable reference to y
    *ry += 10;
    
    assert!(y == 52);
}

()

### Assigning References

* Assigning to a Rust reference makes it point at a new value:

In [24]:
{
    let flag = true;

    let x = 10;
    let y = 20;
    let mut r: &i32 = &x;

    if flag { r = &y; }

    assert!(*r == 10 || *r == 20);
}

()

### References to References

* Rust permits references to references.
* Explicit dereferencing requires as many `*` as levels of indirections
* The `.` operator follows as many references as it takes to find its target.

In [25]:
struct Point { x: i32, y: i32 }

fn ref_to_ref() {
    let point = Point { x: 1000, y: 729 };

    let ry: &i32 = &point.y;
    let rry: &&i32 = &ry;
    let rrry: &&&i32 = &rry;
    assert!(***rrry == 729);

    let r: &Point = &point;
    let rr: &&Point = &r;
    let rrr: &&&Point = &rr;
    assert!(rrr.y == 729);
}

ref_to_ref();

### Comparing References

* Like the `.` operator, Rust’s comparison operators “see through” any number of references, as long as both operands have the same type.
* If you actually want to know whether two references point to the same memory, you can use `std::ptr::eq`, which compares them as addresses:

In [22]:
{
    let x = 10;
    let y = 10;

    let rx = &x;
    let ry = &y;

    assert!(rx == ry);              // their referents are equal
    assert!(!std::ptr::eq(rx, ry)); // but occupy different addresses
    
    let rrx = &rx;
    let rry = &ry;

    assert!(**rrx <= **rry);   // explicit dereferencing using **
    assert!(rrx == rry);       // implicit dereferencing using **
    assert!(rrx <= rry);
}

()

### References Are Never Null
* Rust references are never null. 
  * there’s no analogue to C’s `NULL` or C++’s `nullptr`
  * and Rust won’t convert integers to references (outside of unsafe code), so you can’t convert zero into a reference.

### Borrowing References to Arbitrary Expressions

* Rust lets you borrow a reference (apply `&`) to the value of any sort of expression:

In [28]:
fn factorial(n: usize) -> usize {
    (1..n+1).fold(1, |a, b| a * b)
}

fn main() {
    let r = &factorial(6);

    // Arithmetic operators can see through one level of references.
    assert_eq!(r + &1009, 1729);
}

main();

* Rust simply creates an anonymous variable to hold the expression’s value, and makes the reference point to that. The lifetime of this anonymous variable depends on what you do with the reference:

  * if you immediately assign the reference to a variable in a `let` statement (or make it part of some struct or array that is being immediately assigned), then Rust makes the anonymous variable live as long as the variable the `let` initializes. In the preceding example, Rust would do this for the referent of `r`.
  * Otherwise, the anonymous variable lives to the end of the enclosing statement. In our example, the anonymous variable created to hold `1009` lasts only to the end of the `assert_eq!` statement.

## Reference safety

### Borrowing a Local 

* You can’t borrow a reference to a local variable and take it out of the variable’s scope:

In [29]:
{
    let r;

    {
        let x = 42;
        r = &x;
    }

    assert!(*r == 1);
}

Error: `x` does not live long enough

### Receiving References as Parameters

#### Static variables

* Rust’s equivalent of a global variable is called a **static**
* It’s a value that’s created when the program starts and lasts until it terminates
* Every static must be initialized
* **Mutable statics** are inherently not thread-safe (after all, any thread can access a static at any time), and even in single-threaded programs, they can fall prey to other sorts of reentrancy problems. For these reasons, you may access a mutable static only within an `unsafe` block.


#### Assignment to static reference

* Static references can only refer to other statics - this avoids dangling reference
* In the code below variable `p` has some lifetime (but it is not a static) `=>` Rust rejects this code - assignment `p` to `STASH` could lead to dangling reference 

In [9]:
static mut STASH: &i32 = &128;

fn put_on_stash(p: &i32) {
    unsafe {
        STASH = p;
    }
}

Error: lifetime may not live long enough

* In order to compile `put_on_stash()` function we have annotate that `p` is a reference to static data

In [10]:
static mut STASH: &i32 = &10;

fn put_on_stash(p: &'static i32) {
    unsafe {
        STASH = p;
    }
}

In [11]:
static SOME_DATA: i32 = 665;

put_on_stash(&SOME_DATA);

unsafe { *STASH }

665

### Returning references

* Functions can return references
* Dangling references are reported at compile time

In [14]:
fn smallest(v: &[i32]) -> &i32 {
    let mut smallest_value: &i32 = &v[0];

    for value in &v[1..] {
        if *value < *smallest_value {
            smallest_value = value;
        }    
    }
    smallest_value
}

In [30]:
let v;
{
    let data = [5, 345, 1, 4, 42, 665];
    v = smallest(&data)
}
assert!(*v == 1);

Error: cannot find function `smallest` in this scope

* Correct version of the snippet above

In [31]:
{
    let data = [5, 345, 1, 4, 42, 665];
    let v = smallest(&data);
    assert!(*v == 1);
}

Error: cannot find function `smallest` in this scope